In [3]:
import json
import geojson
from functools import partial
import shapely.geometry
from shapely.geometry import Polygon
import shapely.ops

In [56]:
# reading into two geojson objects, in a GCS (WGS84)
with open('us-states.json') as geojson:
    poly_geojson = json.load(geojson)

elec_regions = {'NW':{'WA','OR','NV','MT','ID','WY','UT','CO'},
               'CAL':{'CA'},
               'SW':{'AZ','NM'},
               'CENT':{'ND','SD','NE','KS','OK'},
               'CAR':{'NC','SC'},
               'FLA':{'FL'},
               'MIDA':{'OH','PA','WV','VA','MD','DE','NJ'},
               'MIDW':{'MN','IA','MO','AR','LA','WI','IL','MI','IN'},
               'NE':{'VT','NH','MA','CT','RI','ME'},
               'NY':{'NY'},
               'SE':{'MS','AL','GA'},
               'TEN':{'KY','TN'},
               'TEX':{'TX'}}

# Verify the total number of states.
tot_states = sum([len(v) for _,v in elec_regions.items()])
print('total number of lower states: {}'.format(tot_states))

elec_regions_reversed = {s:k for k,v in elec_regions.items() for s in v}

total number of lower states: 48


In [57]:
merged_polygons = {k:Polygon() for k,_ in elec_regions.items()}

In [58]:
# Union Polygons
for state in poly_geojson['features']:
    state_abb = state['properties']['name']
    if state_abb in elec_regions_reversed:
        merged_polygons[elec_regions_reversed[state_abb]] = merged_polygons[elec_regions_reversed[state_abb]].union(shapely.geometry.asShape(state['geometry']))

In [64]:
features = []
ind = 1
for k,v in merged_polygons.items():
    features.append({'type':'Feature', 'id':str(ind), 'properties':{'name':k}, 'geometry':shapely.geometry.mapping(v)})

In [65]:
new_geojson = {'type':'FeatureCollection','features':features}
with open('merged_states.json', 'w') as outfile:
   json.dump(new_geojson, outfile)
outfile.close()